In [3]:
import numpy as np
import sys, json, copy, time, pickle, random
from concurrent.futures import ThreadPoolExecutor
import qiskit
from qiskit import IBMQ, QuantumCircuit, execute
from qiskit_aer import StatevectorSimulator, AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error, amplitude_damping_error, ReadoutError
# from qiskit.providers.ibmq.managed import IBMQJobManager, ManagedJobSet
# from qiskit.providers.ibmq.apiconstants import ApiJobShareLevel
from qiskit.quantum_info.operators.symplectic import pauli
# from qubit_map import my_qubit_maps
import matplotlib.pyplot as plt
from scipy.stats import sem, unitary_group
from scipy import sparse
sys.path.append(r"/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/EAB(code from Senrui)/")
import EAB_submit, EAB_process
# import EAB_submit_2 as EAB_submit
# import EAB_process
from statistics import stdev
import itertools
from qiskit.compiler import transpile

from qiskit.quantum_info import Pauli
import math

## Data generation

In [12]:
nqubit=2
circuit = QuantumCircuit(nqubit)
circuit.rxx(math.pi/2,0,1)
  

print(circuit) # print a typical example
# print (type(qiskit.quantum_info.PTM(circuit).data))
PTM=qiskit.quantum_info.PTM(circuit).data
diag=np.diagonal(PTM)
np.set_printoptions(precision=0)
print ("diagonal values:",diag)


     ┌───────────┐
q_0: ┤0          ├
     │  Rxx(π/2) │
q_1: ┤1          ├
     └───────────┘
diagonal values: [1.e+00+0.j 1.e+00+0.j 2.e-16+0.j 2.e-16+0.j 1.e+00+0.j 1.e+00+0.j
 2.e-16+0.j 2.e-16+0.j 2.e-16+0.j 2.e-16+0.j 1.e+00+0.j 1.e+00+0.j
 2.e-16+0.j 2.e-16+0.j 1.e+00+0.j 1.e+00+0.j]


In [9]:
nqubit=2
circuit = QuantumCircuit(nqubit)
circuit.rxx(math.pi/2*1.1,0,1)
  

print(circuit) # print a typical example
# print (type(qiskit.quantum_info.PTM(circuit).data))
PTM=qiskit.quantum_info.PTM(circuit).data
diag=np.diagonal(PTM)
print ("diagonal values:",diag)


     ┌──────────────┐
q_0: ┤0             ├
     │  Rxx(1.7279) │
q_1: ┤1             ├
     └──────────────┘
diagonal values: [ 1.        +0.j  1.        +0.j -0.15643447+0.j -0.15643447+0.j
  1.        +0.j  1.        +0.j -0.15643447+0.j -0.15643447+0.j
 -0.15643447+0.j -0.15643447+0.j  1.        +0.j  1.        +0.j
 -0.15643447+0.j -0.15643447+0.j  1.        +0.j  1.        +0.j]


## Data analysis

Parameters: n = 2, C = 20,  L =  [2, 4, 8, 16, 32]
Measurement based simulation, shots = 1000
Measurement bitflip rate = 0.005000
Total error =  0.01973387502142654
Label / Pauli infidelity / Standard deviation
II 0.0 0.0
XI 3.3201099060020667e-06 3.940270389034985e-05
YI 0.019488812841562786 0.00028890563227579536
ZI 0.019669958269702414 0.00028652808779410747
IX -2.4565815118782197e-05 4.026471940954797e-05
XX -6.963699108464283e-06 6.257924846004339e-05
YX 0.01944293267448005 0.0002919844349367276
ZX 0.01963913063416456 0.00029024763640318583
IY 0.029727975101719095 0.0004288959420880109
XY 0.029694016999577744 0.00044283435439156
YY 0.029555442581295943 0.0005153349804469508
ZY 0.029596647569222534 0.00051482382500689
IZ 0.02982023542175727 0.00042393691577926874
XZ 0.029793584856456 0.000436521858735058
YZ 0.029661448552681935 0.0005110779356022518
ZZ 0.029680024244525094 0.0005115658505006773


Note that, the estimation suffers from some degeneracy. E.g., $\lambda_{IZ}$ and $\lambda_{ZZ}$ cannot be individually addressed. Only their geometric mean is estimated and reported. This issue is detailed in [The learnability of Pauli noise](https://arxiv.org/abs/2206.06362).

Label / Pauli error rates
II 0.9802661249785733
XI 0.004857924649527896
YI 2.4839851629224197e-05
ZI -2.980065682001154e-05
IX 0.009957296894477975
XX 0.004925705828500979
YX 2.4943350088332816e-05
ZX -1.4751988418465478e-05
IY 1.6853337303168736e-05
XY 2.7689817947013085e-06
YY -8.757986503654669e-07
ZY -1.8228942176845564e-06
IZ -3.079801564742635e-05
XZ 2.46329051621641e-06
YZ -2.7910185557838973e-06
ZZ 1.9192098977549588e-06


Similarly, in the estimation $\hat{p}_{XI}=\hat{p}_{XX}=0.0025$ due to degeneracy.

Label / True infidelity/ True Pauli error
II   0.0   0.0
XI   0.0   0.01
YI   0.02   0.0
ZI   0.02   0.0
IX   0.0   0.01
XX   0.0   0.0
YX   0.02   0.0
ZX   0.02   0.0
IY   0.02   0.0
XY   0.02   0.0
YY   0.04   0.0
ZY   0.04   0.0
IZ   0.02   0.0
XZ   0.02   0.0
YZ   0.04   0.0
ZZ   0.04   0.0


II 0.0
XI 3.3201099060020667e-06
YI -0.0005111871584372145
ZI -0.00033004173029758596
IX -2.4565815118782197e-05
XX -6.963699108464283e-06
YX -0.0005570673255199492
ZX -0.0003608693658354416
IY 0.009727975101719095
XY 0.009694016999577743
YY -0.010444557418704058
ZY -0.010403352430777467
IZ 0.009820235421757268
XZ 0.009793584856456001
YZ -0.010338551447318066
ZZ -0.010319975755474907


In [9]:
print (plot_dic)

{'II': 0.0, 'XI': 3.3201099060020667e-06, 'YI': -0.0005111871584372145, 'ZI': -0.00033004173029758596, 'IX': -2.4565815118782197e-05, 'XX': -6.963699108464283e-06, 'YX': -0.0005570673255199492, 'ZX': -0.0003608693658354416, 'IY': 0.009727975101719095, 'XY': 0.009694016999577743, 'YY': -0.010444557418704058, 'ZY': -0.010403352430777467, 'IZ': 0.009820235421757268, 'XZ': 0.009793584856456001, 'YZ': -0.010338551447318066, 'ZZ': -0.010319975755474907}


In [10]:
# plot_c100={}
# plot_c100=plot_dic

In [11]:
# plot_c30={}
# plot_c30=plot_dic

In [12]:
# plot_c20={}
# plot_c20=plot_dic

In [13]:
# plot_c10={}
# plot_c10=plot_dic

In [14]:
# plot_c5={}
# plot_c5=plot_dic

In [15]:
# plot_c3={}
# plot_c3=plot_dic

In [16]:
plt.figure(figsize=(15,10))
plt.plot(list(plot_c100.keys()),list(plot_c100.values()),"*",markersize=10)
plt.plot(list(plot_c30.keys()),list(plot_c30.values()),"*",markersize=10)
plt.plot(list(plot_c20.keys()),list(plot_c20.values()),"*",markersize=10)
plt.plot(list(plot_c10.keys()),list(plot_c10.values()),"*",markersize=10)
plt.plot(list(plot_c5.keys()),list(plot_c5.values()),"*",markersize=10)
# plt.plot(list(plot_c3.keys()),list(plot_c3.values()),"*",markersize=10)
plt.legend(["C=100","C=30","C=20","C=10","C=5"])
plt.ylabel("f_measure - f_true",fontsize=20)
plt.savefig("eps=0.01 single CNOT EAB.pdf",format="pdf",dpi=500)

NameError: name 'plot_c100' is not defined

<Figure size 1080x720 with 0 Axes>

In [ ]:
print (eab_result)